In [1]:
import pandas as pd
import numpy as np

In [2]:
df1 = pd.read_csv('tmdb_5000_credits.csv')

In [3]:
df2 = pd.read_csv('tmdb_5000_movies.csv')

In [4]:
df1.rename(columns={'movie_id':'id'}, inplace=True)

In [5]:
df3 = df2.merge(df1)

In [6]:
df3.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'cast', 'crew'],
      dtype='object')

In [7]:
from ast import literal_eval

In [8]:
features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
    df3[feature] = df3[feature].apply(literal_eval)

In [9]:
df3[['cast', 'crew', 'keywords', 'genres']].head()

,cast,crew,keywords,genres
0,"[{'cast_id': 242, 'character': 'Jake Sully', '...","[{'credit_id': '52fe48009251416c750aca23', 'de...","[{'id': 1463, 'name': 'culture clash'}, {'id':...","[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam..."
1,"[{'cast_id': 4, 'character': 'Captain Jack Spa...","[{'credit_id': '52fe4232c3a36847f800b579', 'de...","[{'id': 270, 'name': 'ocean'}, {'id': 726, 'na...","[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,"[{'cast_id': 1, 'character': 'James Bond', 'cr...","[{'credit_id': '54805967c3a36829b5002c41', 'de...","[{'id': 470, 'name': 'spy'}, {'id': 818, 'name...","[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam..."
3,"[{'cast_id': 2, 'character': 'Bruce Wayne / Ba...","[{'credit_id': '52fe4781c3a36847f81398c3', 'de...","[{'id': 849, 'name': 'dc comics'}, {'id': 853,...","[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam..."
4,"[{'cast_id': 5, 'character': 'John Carter', 'c...","[{'credit_id': '52fe479ac3a36847f813eaa3', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':...","[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam..."


In [10]:
l = df3.loc[0, 'crew']

In [11]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return(i['name'])
    
    else:
        return np.nan

In [12]:
get_director(l)

'James Cameron'

###### Returns the list top 3 elements or entire list; whichever is more

In [13]:
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        
        if len(names) > 3:
            names = names[:3]
        return names
    
    # Return empty list in case of missing/malformed data
    return []

###### Define new director, cast, genres and keywords features that are in a suitable form

In [15]:
df3['director'] = df3['crew'].apply(get_director)

features = ['cast', 'keywords', 'genres']

for feature in features:
    df3[feature] = df3[feature].apply(get_list)

In [20]:
# Print the new features of the first 5 films
df3[['title', 'director', 'cast', 'keywords', 'genres']].head()

,title,director,cast,keywords,genres
0,Avatar,James Cameron,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]","[culture clash, future, space war]","[Action, Adventure, Fantasy]"
1,Pirates of the Caribbean: At World's End,Gore Verbinski,"[Johnny Depp, Orlando Bloom, Keira Knightley]","[ocean, drug abuse, exotic island]","[Adventure, Fantasy, Action]"
2,Spectre,Sam Mendes,"[Daniel Craig, Christoph Waltz, Léa Seydoux]","[spy, based on novel, secret agent]","[Action, Adventure, Crime]"
3,The Dark Knight Rises,Christopher Nolan,"[Christian Bale, Michael Caine, Gary Oldman]","[dc comics, crime fighter, terrorist]","[Action, Crime, Drama]"
4,John Carter,Andrew Stanton,"[Taylor Kitsch, Lynn Collins, Samantha Morton]","[based on novel, mars, medallion]","[Action, Adventure, Science Fiction]"


###### Function to convert all strings to lower case and strip names of spaces

In [25]:
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(' ', '')) for i in x]
    
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(' ', ''))
        else:
            return ''

###### Apply clean_data function to my features

In [27]:
features = ['director', 'cast', 'keywords', 'genres']

for feature in features:
    df3[feature] = df3[feature].apply(clean_data)

###### Creating my metadata soup

In [28]:
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])

df3['soup'] = df3.apply(create_soup, axis=1)

###### Import CountVectorizer and create the count matrix

In [30]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df3['soup'])

###### Compute the Cosine Similarity matrix based on the count_matrix

In [31]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(count_matrix, count_matrix)

###### Reset index of our main DataFrame and construct reverse mapping as before

In [38]:
df3.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director,soup
0,237000000,"[action, adventure, fantasy]",http://www.avatarmovie.com/,19995,"[cultureclash, future, spacewar]",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,"[samworthington, zoesaldana, sigourneyweaver]","[{'credit_id': '52fe48009251416c750aca23', 'de...",jamescameron,cultureclash future spacewar samworthington zo...


In [41]:
df3 = df3.reset_index()

indices = pd.Series(df3.index, index=df3['title'])

###### We can now reuse our get_recommendations() function by passing in the new cosine_sim2 matrix as your second argument.

###### # Function that takes in movie title as input and outputs most similar movies

In [45]:
def get_recommendations(title, cosine_sim=cosine_sim):
    
    # Get the index of the movie that matches the title
    idx = indices[title]
    
    # Get the pairwise similarity scores of all movies with that movie
    similar_scores = list(enumerate(cosine_sim[idx]))
    
    # Sort the movies based on similarity scores in descending order
    similar_scores = sorted(similar_scores, key=lambda x: x[1], reverse=True)
    
    # Get the scores of the 10 most similar movies
    similar_scores = similar_scores[1:11]
    
    # Get the movie indices
    movie_indices = [i[0] for i in similar_scores]
    
    # Return the top 10 most similar movies
    return df3['title'].iloc[movie_indices]

In [46]:
get_recommendations('Avatar', cosine_sim)

206                         Clash of the Titans
71        The Mummy: Tomb of the Dragon Emperor
786                           The Monkey King 2
103                   The Sorcerer's Apprentice
131                                     G-Force
215      Fantastic 4: Rise of the Silver Surfer
466                            The Time Machine
715                           The Scorpion King
1      Pirates of the Caribbean: At World's End
5                                  Spider-Man 3
Name: title, dtype: object

In [47]:
get_recommendations('The Dark Knight Rises', cosine_sim)

65               The Dark Knight
119                Batman Begins
4638    Amidst the Devil's Wings
1196                The Prestige
3073           Romeo Is Bleeding
3326              Black November
1503                      Takers
1986                      Faster
303                     Catwoman
747               Gangster Squad
Name: title, dtype: object

In [49]:
get_recommendations('The Godfather', cosine_sim)

867      The Godfather: Part III
2731      The Godfather: Part II
4638    Amidst the Devil's Wings
2649           The Son of No One
1525              Apocalypse Now
1018             The Cotton Club
1170     The Talented Mr. Ripley
1209               The Rainmaker
1394               Donnie Brasco
1850                    Scarface
Name: title, dtype: object